## Input Analysis

In [25]:
import numpy as np
import scipy.stats as st
import pandas as pd
import datetime as dt
import sys


data = pd.read_csv('TonerItDown.csv')
data = data[data['Time of day']<24]
n = len(data)

#Create rate table for Request generation
emp_rate = data.groupby(data['Time of day'].apply(np.floor)).size() / 60
emp_rate = emp_rate.values
coeffs = np.polyfit(np.arange(3,22),emp_rate[3:22],deg = 2)
fitted_rate = np.zeros(24)
fitted_rate[3:22] = coeffs[0]*np.arange(3,22) ** 2 + coeffs[1]*np.arange(3,22) + coeffs[2]
fitted_rate[[0,1,2,22,23]] = np.mean(emp_rate[[0,1,2,22,23]])
rates = pd.DataFrame(data = fitted_rate,columns = ['fitted rate'])


#Paramaters for Initial Diagnosis Time fitted to the normal distribution 
param_diagnose_a = st.norm.fit(data[data['Request location'].isin
                         (['BC_1','BC_10','BC_4','BC_5','BC_6','BC_7','BC_8'])]['Initial diagnose time'])
param_diagnose_b = st.norm.fit(data[data['Request location'].isin(['BC_2','BC_3','BC_9'])]
                               ['Initial diagnose time'])


# Paramaters for the beta distribution fit for Onsite Repair Time
data_on_site = data['On-site repair time'][data['Needs Replacement?'] != 'yes']
n_on_site = len(data_on_site)
params_repair_beta = st.beta.fit(data_on_site)


#Empirical Probabilities of a call orginating at a BC
p_hat = data.groupby(by = 'Request location')['Initial diagnose time'].size() / n
std = np.sqrt(p_hat * (1-p_hat))
probabilities = p_hat.values.tolist()


# Create Python objects from data given in problem specification
Distances = {
            "BC1":{"BC2": 20,"BC3": 30,"BC4": 45,"BC5": 50,"BC6": 50,"BC7": 60,"BC8": 55,"BC9": 60,"BC10": 70,"Dispatch":45},
            "BC2":{"BC1": 20,"BC3": 10,"BC4": 25,"BC5": 30,"BC6": 50,"BC7": 60,"BC8": 55,"BC9": 60,"BC10": 70,"Dispatch": 45},
            "BC3":{"BC1": 30,"BC2": 10,"BC4": 15,"BC5": 20,"BC6": 40,"BC7": 50,"BC8": 45,"BC9": 50,"BC10": 60,"Dispatch": 35},
            "BC4":{"BC1": 45,"BC2": 25,"BC3": 15,"BC5": 5 ,"BC6": 55,"BC7": 65,"BC8": 60,"BC9": 65,"BC10": 75,"Dispatch": 50},
            "BC5":{"BC1": 50,"BC2": 30,"BC3": 20,"BC4": 5 ,"BC6": 60,"BC7": 70,"BC8": 65,"BC9": 70,"BC10": 80,"Dispatch": 55},
            "BC6":{"BC1": 50,"BC2": 50,"BC3": 40,"BC4": 55,"BC5": 60,"BC7": 10,"BC8": 5 ,"BC9": 10,"BC10": 20,"Dispatch": 25},
            "BC7":{"BC1": 60,"BC2": 60,"BC3": 50,"BC4": 65,"BC5": 70,"BC6": 10,"BC8": 15,"BC9": 20,"BC10": 10,"Dispatch": 35},
            "BC8":{"BC1": 55,"BC2": 55,"BC3": 45,"BC4": 60,"BC5": 65,"BC6": 5 ,"BC7": 15,"BC9": 5 ,"BC10": 15,"Dispatch": 30},
            "BC9":{"BC1": 60,"BC2": 60,"BC3": 50,"BC4": 65,"BC5": 70,"BC6": 10,"BC7": 20,"BC8": 5 ,"BC10": 10,"Dispatch": 35},
            "BC10":{"BC1":70,"BC2": 70,"BC3": 60,"BC4": 75,"BC5": 80,"BC6": 20,"BC7": 10,"BC8": 15,"BC9" : 10,"Dispatch": 45}}

BusinessCenters = ["BC1", "BC2", "BC3", "BC4", "BC5", "BC6", "BC7", "BC8", "BC9", "BC10"]

BC_probabilities = (("BC1", 0.039), ("BC2", 0.082), ("BC3", 0.108), ("BC4", 0.135),
                    ("BC5", 0.118), ("BC6", 0.055), ("BC7", 0.124), ("BC8", 0.058),
                    ("BC9", 0.137), ("BC10", 0.142))

In [26]:
class Request:
    def __init__(self, initialized_time):
        self.location = np.random.choice(BusinessCenters,1,p=probabilities)[0]
        self.status = "Waiting"
        self.initialized_time = None
        self.times = []
        self.assigned_mechanic = None
        self.assigned_van = None
        self.first_req(initialized_time)
    
    #Creates random interarrival time for the request to be created, appends time to times
    def first_req(self,initialized_time):
        tmp = initialized_time + dt.timedelta(minutes=int(np.random.poisson((rates[rates.index == 0]['fitted rate'] * 60), 1)[0]))
        self.initialized_time = tmp
        self.times.append(tmp)
        
    #Check if this repair can be done on-site and update the status accordingly
    def update_waiting_status(self):
        if self.status == "Waiting":
            self.status = np.random.choice(["Onsite","Replace"],1,p=[.4,.6])[0] # fix these probs
    
    # Mechanic is found in sim_main. Assigns the found mechanic to the request
    def assign_mechanic (self, mechanicid) :
        self.assigned_mechanic = mechanicid     
       
    #function that calculates the time required to travel between the customers location and [loc]
    def add_travel_time(self, now, loc) :
        self.times.append(now + dt.timedelta(minutes=Distances[self.location][loc]/60))
        
    # Generate Time for mechanic to diagnose the problem
    def gen_diagnose_time(self,now):
        if self.location in ['BC_2','BC_3','BC_9']:
            self.times.append(now + dt.timedelta(minutes=np.random.normal(param_diagnose_b[0], param_diagnose_b[1])))
        else: 
            self.times.append(now + dt.timedelta(minutes=np.random.normal(param_diagnose_a[0], param_diagnose_a[1])))
    
    def gen_onsite_repair_time(self,now):
        self.times.append(now + dt.timedelta(minutes = np.random.beta(params_repair_beta[0], 
                                            params_repair_beta[1])))
    def gen_van_swap_time(self, now) :
        self.times.append(now + dt.timedelta(minutes= np.random.triangular(10,15,25)))
        
    #Random time for a van to replace the copier
    def gen_van_replace_time(self, now) :
        self.times.append(now + dt.timedelta(minutes= np.random.triangular(20,30,60)))
    
    #Van if found in sim_main. Assigns the found vanid to the request
    def assign_van (self,vanid) :
        self.assigned_van = vanid

        
class Van:
    def __init__(self,id):
        self.loc = 'Dispatch'
        self.busy = False
        self.id = id
        
        
class Mechanic:
    def __init__(self,id):
        self.loc = 'Dispatch'
        self.busy = False
        self.id = id

In [27]:
class Simulation:
    def __init__(self, n_mechanics, n_vans):
        
        self.n_mechanics = n_mechanics
        self.n_vans = n_vans
        self.n_runs = n_runs
        self.mechanics = []
        self.vans = []
        self.open_requests = []
        self.closed_requests = []
        
        for m in range(1,self.n_mechanics):
            self.mechanics.append(Mechanic(m))
        for v in range(1,self.n_vans):
            self.vans.append(Van(v))
          
        current_time = dt.datetime.combine(dt.date.today(), dt.time(hour=0))   # Set hour to begin taking requests
        open_time = dt.datetime.combine(dt.date.today(), dt.time(hour=23))     # Set hour to begin handling requests
        close_time = dt.datetime.combine(dt.date.today(), dt.time(hour=23))    # Set hour to end buisness day
        self.open_requests.append(Request(current_time))

        while(current_time < open_time):
            self.open_requests.append(Request(current_time))
            self.move_first()
            current_time = self.open_requests[0].times[-1]
        
        while(current_time < end_time):
            update_num = len((self.open_requests[0]).times)                # Current step for the request
            
            if(update_num == 1):                                           # We need to move a mechanic
                mech = self.find_nearest("Mechanic")                       # Find nearest free mechanic
                self.open_requests[0].assign_mechanic(mech.id)             # Give the request an assigned mechanic
                self.open_requests[0].add_travel_time(current_time, mech.loc)  # Add the mechanics travel time
                self.open_requests.insert(0,Request(current_time))         # Create a new next request
                self.move_first()
                self.move_first()
                
            elif(update_num == 2):                                         # We need to diagnose the problem
                self.open_requests[0].gen_diagnose_time(current_time)      # Randomly generate diagnosis time
                self.open_requests[0].update_waiting_status()              # Randomly generate repair / replace
                self.move_first()
                
            elif(update_num == 3):                                         # Need to either repair or call van 
                if(self.open_requests[0].status == "Onsite"):              # It's a repair
                    self.open_requests[0].gen_onsite_repair_time(current_time) # Generate a repair time
                else:                                                      # Need to request a van
                    van = self.find_nearest("Van")                         # Find a van
                    self.open_requests[0].assign_van(van.id)               # Assign the van to the request
                    self.open_requests[0].add_travel_time(current_time, van.loc) # Add travel time to site
                    self.free_worker("Mechanic")                           # Mechanic is now done with task
                self.move_first()
                
            elif(update_num == 4):                                         # Finish off the van replacement of  
                if(self.open_requests[0].status == "Onsite"): 
                    self.free_worker("Mechanic")                           # Worker finished repair, now free
                    tmp_request = self.open_requests.pop(0)                # Remove the current request
                    self.closed_requests.append(tmp_request)               # Move to the end of the list, done with it
                else:
                    self.open_requests[0].gen_van_replace_time(current_time)
                    self.move_first()
                    
            elif(update_num == 5): # Wait for van to finish 
                self.open_requests[0].add_travel_time(current_time, 'Dispatch')
                self.move_first()
                
            elif(update_num == 6):
                self.open_requests[0].gen_van_swap_time(current_time)
                self.move_first()
                
            elif(update_num == 7):
                tmp = self.open_requests.pop(0)
                self.closed_requests.append(tmp)
                self.free_worker("Van")
                
            current_time = self.open_requests[0].times[-1]
            
            
            
    # Find the nearest free worker (of given type) of out of list and set to busy
    def find_nearest(self, worker_type):
        closest_distance = 100
        assigned_worker = None
        request_loc = self.open_requests[0].location      #location of the request
        
        loop_list = self.mechanics if (worker_type == "Mechanics") else self.vans
       
        for worker in loop_list:
            if(worker.busy == True): pass
            elif(Distances[request_loc][worker.loc] < closest_distance):
                assigned_worker = worker
                closest_distance = Distances[request_loc][worker.loc]
               
        try:
            assigned_worker.busy = True
        except(AttributeError):
            print("No Free worker to assign....Simulation Failed")
            sys.exit()
        return assigned_worker
        

    def free_worker(self, worker_type):
        if(worker_type == "Mechanic"):
            for mechanic in self.mechanics:
                if(mechanic.id == self.open_requests[0].assigned_mechanic): mechanic.busy = False
                    
        elif(worker_type == "Van"):
            for van in self.vans:
                if(van.id == self.open_requests[0].assigned_van): van.busy = False

    def move_first(self):
        if(len(self.open_requests) > 1):
            tmp_req = self.open_requests.pop(0)
            index = 0
            try: 
                while(self.open_requests[index].times[-1] < tmp_req.times[-1]): index += 1
                self.open_requests.insert(index-1, tmp_req)
            except(IndexError):
                self.open_requests.append(tmp_req)
  

In [53]:
n_runs = 10
max_mechanics = 51
max_vans = 51
results = pd.DataFrame()

# Run simulation n times, storing the results in a df
for m in range(40,max_mechanics):
    for v in range(40, max_vans):
        for x in range(1, n_runs):
            sim = Simulation(m, v)
            df = pd.DataFrame([y.times for y in sim.closed_requests])
            df['status'] = pd.DataFrame([y.status for y in sim.closed_requests])
            df['Run'] = x
            df['n_Mechanics'] = m
            df['n_Vans'] = v
            df['Response'] = df[1] - df[0]
            df['Replace'] = df[3] - df[1]
            results = results.append(df)  
        results.to_csv("~/School/ORIE4580/ORIE4580/data/" + str(m) + ", " + str(v) + ".csv")

In [30]:

    
    
    
    
current_time = dt.datetime.combine(dt.date.today(), dt.time(hour=0))
        end_time = dt.datetime.combine(dt.date.today(), dt.time(hour=23))
        self.open_requests.append(Request(current_time))

NameError: name 'self' is not defined

In [54]:
df = pd.read_csv("~/School/ORIE4580/ORIE4580/data/41, 41.csv")
df

,Unnamed: 0,0,1,2,3,4,5,6,status,Run,n_Mechanics,n_Vans,Response,Replace
0,0,2019-12-02 01:08:00.000000,2019-12-02 00:00:55.000000,2019-12-02 00:16:06.674511,2019-12-02 00:16:24.511386,NaN,NaN,NaN,Onsite,1,40,40,-1 days +22:52:55.000000000,0 days 00:15:29.511386000
1,1,2019-12-02 00:46:00.000000,2019-12-02 00:46:35.000000,2019-12-02 01:05:51.354191,2019-12-02 01:06:05.377850,NaN,NaN,NaN,Onsite,1,40,40,0 days 00:00:35.000000000,0 days 00:19:30.377850000
2,2,2019-12-02 02:17:00.000000,2019-12-02 02:17:35.000000,2019-12-02 02:33:34.843402,2019-12-02 02:34:09.843402,2019-12-02 03:12:49.548322,2019-12-02 03:13:24.548322,2019-12-02 03:28:36.647878,Replace,1,40,40,0 days 00:00:35.000000000,0 days 00:16:34.843402000
3,3,2019-12-02 03:38:00.000000,2019-12-02 03:38:30.000000,2019-12-02 03:52:57.528624,2019-12-02 03:53:27.528624,2019-12-02 04:22:08.853466,2019-12-02 04:22:38.853466,2019-12-02 04:35:23.161454,Replace,1,40,40,0 days 00:00:30.000000000,0 days 00:14:57.528624000
4,4,2019-12-02 04:45:00.000000,2019-12-02 04:45:30.000000,2019-12-02 05:03:03.308474,2019-12-02 05:03:15.714860,NaN,NaN,NaN,Onsite,1,40,40,0 days 00:00:30.000000000,0 days 00:17:45.714860000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331,14,2019-12-02 16:14:00.000000,2019-12-02 16:14:45.000000,2019-12-02 16:29:44.001844,2019-12-02 16:30:03.123404,NaN,NaN,NaN,Onsite,9,41,41,0 days 00:00:45.000000000,0 days 00:15:18.123404000
2332,15,2019-12-02 17:30:00.000000,2019-12-02 17:30:30.000000,2019-12-02 17:51:50.477670,2019-12-02 17:52:20.477670,2019-12-02 18:31:54.892395,2019-12-02 18:32:24.892395,2019-12-02 18:48:30.787393,Replace,9,41,41,0 days 00:00:30.000000000,0 days 00:21:50.477670000
2333,16,2019-12-02 18:41:00.000000,2019-12-02 18:41:50.000000,2019-12-02 19:04:05.361326,2019-12-02 19:04:55.361326,2019-12-02 19:37:06.000849,2019-12-02 19:37:56.000849,2019-12-02 19:51:12.450175,Replace,9,41,41,0 days 00:00:50.000000000,0 days 00:23:05.361326000
2334,17,2019-12-02 20:00:00.000000,2019-12-02 20:00:35.000000,2019-12-02 20:17:46.446322,2019-12-02 20:18:21.772100,NaN,NaN,NaN,Onsite,9,41,41,0 days 00:00:35.000000000,0 days 00:17:46.772100000
